## Load the data set

In [ ]:
from xautoml.util.config import plot_runhistory
from xautoml.util.datasets import hearts

X_train, y_train = hearts('../heart.csv', train=True)
X_train

## Start the Model Building

In [ ]:
import pickle
import autosklearn.classification

automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=300,
    per_run_time_limit=60,
    tmp_folder='./output/autosklearn',
    max_models_on_disc=None,
    delete_tmp_folder_after_terminate=False
)
automl.fit(X_train, y_train, dataset_name='hearts')

with open(f'./output/autosklearn.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [ ]:
import pickle

with open(f'./output/autosklearn.pkl', 'rb') as f:
    automl = pickle.load(f)

## Get the Score of the Final Ensemble

In [ ]:
from sklearn.metrics import accuracy_score

X_test, y_test = hearts('../heart.csv', test=True)

predictions = automl.predict(X_test)
accuracy_score(y_test, predictions)

## Show Performance of All Evaluated Configurations over Time

In [ ]:
import numpy as np

plot_data = np.array(list(map(lambda v: (v.endtime, 1 - v.cost), automl.automl_.runhistory_.data.values())))
plot_runhistory(plot_data)

## View the Models found by auto-sklearn

In [ ]:
automl.leaderboard()

## Print the Final Ensemble Constructed by auto-sklearn

In [ ]:
automl.show_models()

## Load the Same Results in XAutoML

In [ ]:
from xautoml.main import XAutoML
from xautoml.adapter import import_auto_sklearn

rh = import_auto_sklearn(automl)
main = XAutoML(rh, X_test, y_test)

main